# 播放音频文件

由于安全方面的原因，你并不能通过 JupyterLab 来直接访问音频设备（环境的限制），我们这里的代码块不供用户运行。

这里的程序来自于产品主程序的 audio_ctrl.py，你可以参考这里的代码来了解产品主程序是如何实现音频文件播放功能的。

## 产品主程序与音频播放相关的功能

产品主程序的文件夹内有一个名称为 sounds 的文件夹，这个文件夹内有很多子文件夹：connected、others、recv_new_cmd、robot_started、searching_for_target、target_detected、target_locked。

在我们提供的默认程序中，只有分别在 connected 和 robot_started 中各放置了一个音频文件。

当机器人主程序运行起来后，会自动随机播放一个 robot_started 内的音频文件。

当有客户端使用浏览器连接到这个 WEB 应用时，会自动随机播放一个 connected 内的音频文件。

你可以在这些文件夹内放入自定义的音频文件当作语音包，来让你的产品更加定制化。

In [ ]:
import pygame  # 导入pygame库，用于音频播放
import random  # 导入random库，用于随机选择音频文件
import yaml  # 导入yaml库，用于读取配置文件
import os  # 导入os库，用于文件操作
import threading  # 导入threading库，用于多线程处理

# 获取配置文件
curpath = os.path.realpath(__file__)  # 获取当前脚本的绝对路径
thisPath = os.path.dirname(curpath)  # 获取当前脚本所在目录
with open(thisPath + '/config.yaml', 'r') as yaml_file:  # 打开配置文件
    config = yaml.safe_load(yaml_file)  # 使用yaml库加载配置文件
    
# 初始化pygame.mixer，设置音频输出的默认音量
pygame.mixer.init()
pygame.mixer.music.set_volume(config['audio_config']['default_volume'])

# 创建一个事件对象，用于控制音频播放
play_audio_event = threading.Event()

# 从配置文件中读取最小播放间隔时间
min_time_bewteen_play = config['audio_config']['min_time_bewteen_play']

# 定义播放音频的函数
def play_audio(input_audio_file):
    try:
        pygame.mixer.music.load(input_audio_file)  # 加载音频文件
        pygame.mixer.music.play()  # 播放音频
    except:
        play_audio_event.clear()  # 出错时清除事件标志
        return
    while pygame.mixer.music.get_busy():  # 等待音频播放完成
        pass
    time.sleep(min_time_bewteen_play)  # 等待最小播放间隔时间
    play_audio_event.clear()  # 清除事件

# 定义播放随机音频的函数
def play_random_audio(input_dirname, force_flag):
    if play_audio_event.is_set() and not force_flag:
        return
    # 获取指定目录下的所有音频文件
    audio_files = [f for f in os.listdir(current_path + "/sounds/" + input_dirname) if f.endswith((".mp3", ".wav"))]
    # 从音频文件列表中随机选择一个音频文件
    audio_file = random.choice(audio_files)
    play_audio_event.set()  # 设置事件
    # 创建一个线程来播放音频
    audio_thread = threading.Thread(target=play_audio, args=(current_path + "/sounds/" + input_dirname + "/" + audio_file,))
    audio_thread.start()  # 启动线程

# 定义播放音频的线程函数
def play_audio_thread(input_file):
    if play_audio_event.is_set():  # 如果事件已经设置，则返回
        return
    play_audio_event.set()  # 设置事件
    # 创建一个线程来播放音频
    audio_thread = threading.Thread(target=play_audio, args=(input_file,))
    audio_thread.start()  # 启动线程

# 定义播放指定文件的函数
def play_file(audio_file):
    audio_file = current_path + "/sounds/" + audio_file # 构建音频文件的完整路径
    play_audio_thread(audio_file) # 在新线程中播放音频

# 定义设置音频音量的函数
def set_audio_volume(input_volume):
    input_volume = float(input_volume)  # 将输入音量转换为浮点数
    if input_volume > 1:  # 如果音量大于1，则设置为1
        input_volume = 1
    elif input_volume < 0:  # 如果音量小于0，则设置为0
        input_volume = 0
    pygame.mixer.music.set_volume(input_volume)  # 设置音量

# 定义设置最小播放间隔时间的函数
def set_min_time_between(input_time):
    global min_time_bewteen_play  # 使用全局变量
    min_time_bewteen_play = input_time  # 设置最小播放间隔时间